In [1]:
import imp
import pandas as pd

# Generate overall summary table - 2017

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway, and for the four OSPAR regions. This notebook reads the output from the processing of the observed data (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_working_2016-17.ipynb)) and the modelling using NOPE (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions_2016.ipynb)) and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

In [2]:
# Import custom RID functions
rid_func_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
                 r'\Python\rid\notebooks\useful_rid_code.py')

rid = imp.load_source('useful_rid_code', rid_func_path)

# Import model
nope_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\Python\rid\notebooks\nope.py')
nope = imp.load_source('nope', nope_path)

## 1. Summarise monitoring data

In [3]:
# Read station data
in_xlsx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Data\RID_Sites_List_2017-2020.xlsx')
stn_df = pd.read_excel(in_xlsx, sheet_name='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=['ospar_region', 'nve_vassdrag_nr'])

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\NOPE_Annual_Inputs\nope_input_data_1990.csv')
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

**Note:** Remember to update the year below.

In [4]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\loads_and_flows_all_sites_2017.csv')
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
#mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'new_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('new_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'new_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id']

# Rename cols to match template
mon_df['Flow rate_1000m3/day'] = mon_df['mean_q_1000m3/day']
del mon_df['mean_q_1000m3/day']

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr    Cu    Hg   NH4-N  \
ospar_region        new_rid_group                                             
LOFOTEN-BARENTS SEA rid_135        0.0   1.0  0.0   2.0   9.0   8.0   113.0   
                    rid_20         0.0   2.0  0.0   5.0  27.0  12.0   218.0   
NORTH SEA           rid_135        0.0   3.0  0.0   4.0  22.0  31.0   345.0   
                    rid_20         0.0   1.0  0.0   0.0   2.0   4.0    26.0   
NORWEGIAN SEA2      rid_135        0.0   4.0  0.0   8.0  29.0  29.0   349.0   
                    rid_20         0.0   1.0  0.0   2.0  17.0   1.0    21.0   
SKAGERAK            rid_135        0.0   3.0  0.0   2.0   9.0  13.0   202.0   
                    rid_20         0.0   9.0  1.0   9.0  50.0  49.0   592.0   
NORWAY              rid_135        0.0  12.0  1.0  15.0  69.0  82.0  1009.0   
                    rid_20         0.0  12.0  1.0  16.0  96.0  66.0   857.0   

                                     NO3-N    Ni  PO4-P    Pb       SPM  \
ospar_region        new_rid_group                                         
LOFOTEN-BARENTS SEA rid_135          403.0   7.0   21.0   0.0   19889.0   
                    rid_20           550.0  56.0  122.0   1.0   60274.0   
NORTH SEA           rid_135         7063.0  12.0   78.0   7.0   67712.0   
                    rid_20          1182.0   2.0   19.0   1.0    6605.0   
NORWEGIAN SEA2      rid_135         3010.0  23.0   86.0   2.0  115688.0   
                    rid_20          1242.0   6.0   20.0   0.0   22864.0   
SKAGERAK            rid_135         2776.0   5.0   31.0   4.0   27025.0   
                    rid_20         13579.0  28.0  316.0  10.0  266364.0   
NORWAY              rid_135        13251.0  46.0  215.0  14.0  230314.0   
                    rid_20         16554.0  92.0  478.0  12.0  356108.0   

                                       SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135         37196.0   34843.0   1763.0   38.0   10.0   
                    rid_20         100369.0   79114.0   4232.0  228.0   26.0   
NORTH SEA           rid_135         65604.0   75522.0  12343.0  229.0   84.0   
                    rid_20           7979.0   10822.0   1869.0   47.0   10.0   
NORWEGIAN SEA2      rid_135         81891.0   83347.0   8143.0  209.0   67.0   
                    rid_20          25903.0   25876.0   2327.0   46.0   24.0   
SKAGERAK            rid_135         29882.0   64462.0   5786.0  119.0   58.0   
                    rid_20         165066.0  211721.0  23060.0  576.0  128.0   
NORWAY              rid_135        214573.0  258175.0  28035.0  594.0  218.0   
                    rid_20         299317.0  327532.0  31488.0  897.0  188.0   

                                   Flow rate  
ospar_region        new_rid_group             
LOFOTEN-BARENTS SEA rid_135          45661.0  
                    rid_20           58352.0  
NORTH SEA           rid_135         150488.0  
                    rid_20           17801.0  
NORWEGIAN SEA2      rid_135         130472.0  
                    rid_20           32480.0  
SKAGERAK            rid_135          36733.0  
                    rid_20          149456.0  
NORWAY              rid_135         363353.0  
                    rid_20          258089.0

## 2. Data for unmonitored areas

**Note:** Remember to update the year below.

In [5]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Unmon_loads\unmon_loads_2017.csv')
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,507481.0,13971.0,1074.0,2471.0,290.0,59962.0,10350.0,39601.0,771.0,645.0,...,13637.0,0.0,0.0,0.0,5.0,20.0,3.0,1.0,10.0,1088.0
LOFOTEN-BARENTS SEA,138847.0,1205.0,163.0,81.0,6.0,15814.0,2731.0,6359.0,125.0,98.0,...,2357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,286.0
NORTH SEA,174898.0,3735.0,432.0,529.0,77.0,19137.0,3299.0,15986.0,253.0,259.0,...,5429.0,0.0,0.0,0.0,2.0,9.0,1.0,1.0,4.0,346.0
NORWEGIAN SEA2,183121.0,2735.0,351.0,911.0,158.0,24982.0,4314.0,14197.0,305.0,211.0,...,3531.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,455.0
SKAGERAK,10615.0,6296.0,128.0,951.0,50.0,29.0,5.0,3059.0,88.0,77.0,...,2319.0,0.0,0.0,0.0,2.0,9.0,1.0,0.0,2.0,1.0


## 3. Create table

The cell below uses the 3-fold division of sites (`'rid_11'` etc.) from the old progreamme structure. To use this, copy the old Word template and make sure that the aggregation in Section 1 is done by `'old_rid_group'`.

In [ ]:
# Create table 3 (old programme structure)
in_docx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Results\Word_Tables\2018Analysis_2017Data\Table_3_2017.docx')

rid.write_word_overall_table(mon_df, umon_df, in_docx)

The cell below uses the 2-fold division of sites (`'rid_20'` and `'rid_135'`) for the 2017-20 progreamme structure. To use this, copy the new Word template and make sure that the aggregation in Section 1 is done by `'new_rid_group'`.

In [6]:
# Create table 3
in_docx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Results\Word_Tables\2018Analysis_2017Data\Table_3_2017_new.docx')

rid.write_word_overall_table_2017_20(mon_df, umon_df, in_docx)

Finished.
